In [47]:
#importer les librairies
import pandas as pd
import re
import sqlparse
import glob
import warnings
import os



In [48]:

#lire toutes les infos du df
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

## Utilisation d'un fichier excel

In [52]:
#créer le dataframe à partir d'un fichier csv
df=pd.read_excel("FORMATION_EX_FINAL_query.xlsx")

c:\Python\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Utilisation d'un répertoire

In [4]:
"""
# Chemin du répertoire contenant les fichiers
chemin_repertoire = "C:/Talend/TOS_DI-8.0.1/studio/workspace/"

# Modèle pour les fichiers que vous souhaitez lire
modele_fichier = "*_query.xlsx"

# Utiliser glob pour obtenir la liste des fichiers correspondants au modèle
chemin_fichiers = glob.glob(chemin_repertoire + modele_fichier)

# Initialiser un DataFrame vide pour stocker les données de tous les fichiers
df = pd.DataFrame()

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")

# Boucle pour lire chaque fichier, ajouter une colonne avec le nom du fichier, et concaténer les données
    for fichier in chemin_fichiers:
        df_temp = pd.read_excel(fichier)
        
        # Extraire le nom du fichier (sans le chemin)
        nom_fichier = fichier.split("\\")[-1]
        nom_fichier= nom_fichier.split(".")[0]
        
        # Ajouter une colonne "Nom_Fichier" au DataFrame temporaire
        df_temp = df_temp.assign(projet=nom_fichier)
        
        # Concaténer les données dans le DataFrame global
        df = pd.concat([df, df_temp], ignore_index=True)

# Supprimer tous les fichiers Excel après le traitement
for fichier in chemin_fichiers:
    os.remove(fichier)
    print(f'Le fichier {fichier} a été supprimé.')

print("Tous les fichiers Excel ont été traités et supprimés.")
"""

'\n# Chemin du répertoire contenant les fichiers\nchemin_repertoire = "C:/Talend/TOS_DI-8.0.1/studio/workspace/"\n\n# Modèle pour les fichiers que vous souhaitez lire\nmodele_fichier = "*_query.xlsx"\n\n# Utiliser glob pour obtenir la liste des fichiers correspondants au modèle\nchemin_fichiers = glob.glob(chemin_repertoire + modele_fichier)\n\n# Initialiser un DataFrame vide pour stocker les données de tous les fichiers\ndf = pd.DataFrame()\n\nwith warnings.catch_warnings(record=True):\n    warnings.simplefilter("always")\n\n# Boucle pour lire chaque fichier, ajouter une colonne avec le nom du fichier, et concaténer les données\n    for fichier in chemin_fichiers:\n        df_temp = pd.read_excel(fichier)\n        \n        # Extraire le nom du fichier (sans le chemin)\n        nom_fichier = fichier.split("\\")[-1]\n        nom_fichier= nom_fichier.split(".")[0]\n        \n        # Ajouter une colonne "Nom_Fichier" au DataFrame temporaire\n        df_temp = df_temp.assign(projet=nom_

In [50]:
#créer les fonctions regex
def extraire_noms_tables(requete_sql):
    # Utilisation d'une expression régulière pour trouver les occurrences de motifs de nom de table
    motif = r'\bFROM\s+([a-zA-Z_][a-zA-Z0-9_]*)\b|\bJOIN\s+([a-zA-Z_][a-zA-Z0-9_]*)\b|\bINTO\s+([a-zA-Z_][a-zA-Z0-9_]*)\b'
    matches = re.findall(motif, requete_sql, flags=re.IGNORECASE)

    # Regrouper les résultats de toutes les expressions régulières
    noms_tables = [table for match in matches for table in match if table]

    return noms_tables

def extraire_noms_tables(requete_sql):
    # Utilisation d'une expression régulière pour trouver les occurrences de motifs de nom de table
    motif = r'\bFROM\s+([a-zA-Z_][a-zA-Z0-9_]*)\b|\bJOIN\s+([a-zA-Z_][a-zA-Z0-9_]*)\b|\bINTO\s+([a-zA-Z_][a-zA-Z0-9_]*)\b'
    matches = re.findall(motif, requete_sql, flags=re.IGNORECASE)

    # Regrouper les résultats de toutes les expressions régulières
    noms_tables = [table for match in matches for table in match if table]

    return noms_tables

def extract_old_sql(requete_sql):
    # Recherche de l'indice du mot "FROM"
    index_from = requete_sql.upper().find("FROM")
    
    if index_from != -1:
        # Extrait la sous-chaîne après le mot "FROM"
        sous_chaine = requete_sql[index_from + 4:]
        
        # Supprime les espaces en début et en fin de chaîne
        sous_chaine = sous_chaine.strip()
        
        # Recherche de l'indice du mot "WHERE"
        index_where = sous_chaine.upper().find("WHERE")
        
        if index_where != -1:
            # Extrait la sous-chaîne avant le mot "WHERE"
            sous_chaine_tables = sous_chaine[:index_where].strip()
            
            # Vérifie s'il y a une virgule dans la sous-chaîne
            if ',' in sous_chaine_tables:
                # Sépare les noms de table en fonction des virgules
                noms_tables = [table.strip() for table in sous_chaine_tables.split(',')]
                return noms_tables
            
def mettre_none_si_commence_par_parenthese(valeur):
    if isinstance(valeur, str) and valeur.startswith("("):
        return None
    else:
        return valeur

In [55]:
df.head(20)

,projet,nom_fichier,nom_fichier_sans_extension,component_name,component_value,requete_sql,id_job,label_job,version
0,FORMATION_EX_FINAL,Connexion_BDD_et_commit_0.1.item,Connexion_BDD_et_commit_01,tPostgresqlInput,tDBInput_1,,_PzxOkId-Ee6J8YvAz2v8Ng,Connexion_BDD_et_commit,0.1


In [54]:
# Supprimer les requêtes inutiles
df['requete_sql']= df['requete_sql'].replace("select id  name from employee", "requete nom modifiee")

In [16]:
# Lancement du traitement

# Appliquer la fonction personnalisée à chaque ligne de la colonne 'requete_sql'
df['table'] = df['requete_sql'].apply(extraire_noms_tables)

# S'assurer que la colonne 'table' est de type chaîne
df['table'] = df['table'].astype(str)

# Supprimer les crochets de la colonne 'table'
df['table']=df['table'].str.replace('[', '').str.replace(']', '')

# Supprimer tous les guillemets simples de la colonne 'table'
df['table'] = df['table'].str.replace('\'', '')

In [17]:


# Appliquer la fonction personnalisée à chaque ligne de la colonne 'requete_sql'
df['table_2'] = df['requete_sql'].apply(extract_old_sql)

# S'assurer que la colonne 'table' est de type chaîne
df['table_2'] = df['table_2'].astype(str)

# Supprimer les crochets de la colonne 'table'
df['table_2']=df['table_2'].str.replace('[', '').str.replace(']', '')

# Supprimer tous les guillemets simples de la colonne 'table'
df['table_2'] = df['table_2'].str.replace('\'', '')

# Appliquer la fonction à la colonne 'table_2'
df['table_2'] = df['table_2'].apply(mettre_none_si_commence_par_parenthese)

# Remplacer les valeurs nulles (None) dans la colonne "table_2" par celles de la colonne "table"
df['table_2'] = df['table_2'].apply(lambda x: x if x != 'None' else None)
df['table_2'].fillna(df['table'], inplace=True)

# Supprimer la colonne "table"
df.drop('table', axis=1, inplace=True)


In [72]:
df.sample(5)

,projet,nom_fichier,nom_fichier_sans_extension,component_name,component_value,requete_sql,id_job,label_job,version,table_2
2,FORMATION_TALEND,Exercice_5_suite_to_Database_0.1.item,Exercice_5_suite_to_Database_01,tPostgresqlInput,tDBInput_1,SELECT \n table_exercice_3 id \n table_ex...,NaN,NaN,NaN,table_exercice_3
4,FORMATION_TALEND,Exercice_7_0.1.item,Exercice_7_01,tPostgresqlInput,tDBInput_1,SELECT \n tables_exercice_5 id \n tables_...,NaN,NaN,NaN,tables_exercice_5
1,FORMATION_TALEND,Exercice_5_0.1.item,Exercice_5_01,tPostgresqlInput,tDBInput_1,SELECT \n table_exercice_3 id \n table_ex...,NaN,NaN,NaN,table_exercice_3
3,FORMATION_TALEND,Exercice_6_0.1.item,Exercice_6_01,tPostgresqlInput,tDBInput_1,SELECT \n table_exercice_3 id \n table_ex...,NaN,NaN,NaN,table_exercice_3
0,FORMATION_TALEND,Exercice_4_0.1.item,Exercice_4_01,tPostgresqlInput,tDBInput_1,SELECT \n table_exercice_3 id \n table_exer...,NaN,NaN,NaN,table_exercice_3


In [9]:
#Supprimer les retours à la ligne
for column in df.columns:
    df[column] = df[column].astype(str).str.replace('\n', ' ')
    df[column] = df[column].astype(str).str.replace('\r', ' ')
    
# Remplacer les virgules par des points virgules
df['table_2'] = df['table_2'].astype(str).str.replace(',', ';')

# Diviser les valeurs de la colonne contenant des points-virgules
df['table_2'] = df['table_2'].str.split(';')

# Utiliser la fonction explode pour déplier les listes résultantes
df = df.explode('table_2')

df.head(80)

,projet,nom_fichier,nom_fichier_sans_extension,component_name,component_value,requete_sql,id_job,label_job,version,table_2
0,FORMATION_TALEND,Exercice_4_0.1.item,Exercice_4_01,tPostgresqlInput,tDBInput_1,SELECT table_exercice_3 id table_exerci...,nan,nan,nan,table_exercice_3
1,FORMATION_TALEND,Exercice_5_0.1.item,Exercice_5_01,tPostgresqlInput,tDBInput_1,SELECT table_exercice_3 id table_exer...,nan,nan,nan,table_exercice_3
2,FORMATION_TALEND,Exercice_5_suite_to_Database_0.1.item,Exercice_5_suite_to_Database_01,tPostgresqlInput,tDBInput_1,SELECT table_exercice_3 id table_exer...,nan,nan,nan,table_exercice_3
3,FORMATION_TALEND,Exercice_6_0.1.item,Exercice_6_01,tPostgresqlInput,tDBInput_1,SELECT table_exercice_3 id table_exer...,nan,nan,nan,table_exercice_3
4,FORMATION_TALEND,Exercice_7_0.1.item,Exercice_7_01,tPostgresqlInput,tDBInput_1,SELECT tables_exercice_5 id tables_ex...,nan,nan,nan,tables_exercice_5
5,FORMATION_TALEND,Exercice_8b_0.1.item,Exercice_8b_01,tPostgresqlInput,tDBInput_1,SELECT table_exercice_3 id table_exer...,nan,nan,nan,table_exercice_3
6,FORMATION_TALEND,Final_Exercice_0.1.item,Final_Exercice_01,tPostgresqlInput,tDBInput_2,SELECT table_exercice_3 id table_exerci...,nan,nan,nan,table_exercice_3


In [10]:
df.to_csv("test_tables_sql.csv", index=False)

In [11]:
#df.to_csv("C:/Talend/TOS_DI-8.0.1/studio/workspace/job_tables_sql.csv", index=False)